In [1]:
import pandas as pd
import os, glob
import re
import shutil
import pyodbc
import numpy as np
#from simpledbf import Dbf5


In [ ]:
#Lookup_project_type.head()

In [3]:
pd.set_option('display.max_columns', None)


In [5]:
Tdf = df

#Tdf= Tdf[Tdf['PHExits']=='Yes']
Tdf = Tdf[['AcuityScore','Veteran','HoH','ProgramTypeCode','Gender','ethnicity','Age','race','active','NewEnrolled','PHExits']]
Tdf = Tdf[Tdf['race']!='Multirace']
#Conditions----

# Head of Household-----
Tdf['HoH'] = np.where(Tdf['HoH']=='Yes', 1, 2)
#Tdf = Tdf.drop('HoH',axis=1)
#--------

#---Gender-----
conditionsGender = [
    (Tdf['Gender'] == 'Female') ,
    (Tdf['Gender'] == 'Male') ,
    (Tdf['Gender'] == 'Trans Female (FTM or Male to Female)'),
    (Tdf['Gender'] == 'Trans Male (FTM or Female to Male)'),
    (Tdf['Gender'] == 'Gender Non-Conforming (i.e. not exclusively male or female)'),
    (Tdf['Gender'] == "Client doesn't know"),
    (Tdf['Gender'] == "Client refused")]
choicesGender = [0,1,2,3,4,8,9]
Tdf['Gender'] = np.select(conditionsGender, choicesGender, default= 99)
#-------------

# Active: 1 for active and 2 not active.----------
Tdf['active'] = np.where(Tdf['active']=='Yes', 1, 2) 

# New enrolled: 1 for yes and 2 for now
Tdf['NewEnrolled'] = np.where(Tdf['NewEnrolled']=='Yes', 1, 2)

# Ethnicity: 
conditionsEthnicity = [
    (Tdf['ethnicity'] == 'Non-Hispanic/Non-Latino') ,
    (Tdf['ethnicity'] == 'Hispanic/Latino') ,
    (Tdf['ethnicity'] == "Client doesn't know"),
    (Tdf['ethnicity'] == "Client refused")]
choicesEthnicity = [1,2,8,9]
Tdf['ethnicity'] = np.select(conditionsEthnicity, choicesEthnicity, default= 99)

# VETERAN
#Tdf['Veteran'] = np.where(Tdf['Veteran']=='Yes', 1, 2) 
conditionsVeteran = [
    (Tdf['Veteran'] == 'No') ,
    (Tdf['Veteran'] == 'Yes') ,
    (Tdf['Veteran'] == "Client doesn't know"),
    (Tdf['Veteran'] == "Client refused")]
choicesVeteran = [0,2,8,9]
Tdf['Veteran'] = np.select(conditionsVeteran, choicesVeteran, default= 99)

#Race
conditionsRace = [
    (Tdf['race'] == 'American Indian or Alaska Native') ,
    (Tdf['race'] == 'Asian') ,
    (Tdf['race'] == "Black or African American"),
    (Tdf['race'] == "Native Hawaiian or Other Pacific Islander"),
    (Tdf['race'] == "White"),
    (Tdf['race'] == "Client doesn't know"),
    (Tdf['race'] == "Client refused")]
choicesRace = [1,2,3,4,5,8,9]
Tdf['race'] = np.select(conditionsRace, choicesRace, default= 99)

#PH Exits
Tdf['PHExits'] = np.where(Tdf['PHExits']=='Yes', 1, 2) 
Tdf.head()

,AcuityScore,Veteran,HoH,ProgramTypeCode,Gender,ethnicity,Age,race,active,NewEnrolled,PHExits
0,9,0,1,13,0,2,25,5,2,1,1
1,9,0,1,1,0,2,25,5,2,1,2
2,9,0,1,1,0,2,25,5,2,1,1
3,4,0,1,13,0,1,32,3,1,1,2
4,4,0,1,1,0,1,32,3,2,1,1


In [13]:
Tdf['PHExits'].value_counts()

2    2868
1     579
Name: PHExits, dtype: int64

In [17]:
from sklearn.svm import SVC

In [18]:
X = Tdf.drop('PHExits', axis=1)
y = Tdf['PHExits']

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [20]:
#SVM - Polynomial Kernel
svclassifier = SVC(kernel='poly', degree=4)
svclassifier.fit(X_train, y_train)

D:\Felix\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=4, gamma='auto_deprecated',
    kernel='poly', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [21]:
#Making Predictions
y_pred = svclassifier.predict(X_test)

In [22]:
#Evaluating the Algorithm

from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[ 53  67]
 [ 41 529]]
              precision    recall  f1-score   support

           1       0.56      0.44      0.50       120
           2       0.89      0.93      0.91       570

    accuracy                           0.84       690
   macro avg       0.73      0.68      0.70       690
weighted avg       0.83      0.84      0.84       690



In [24]:
# SVM using a linear kernel
svclassifier2 = SVC(kernel='linear')
svclassifier2.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [25]:
y_predLinear = svclassifier2.predict(X_test)

In [27]:
print(confusion_matrix(y_test, y_predLinear))
print(classification_report(y_test, y_predLinear))

[[ 24  96]
 [ 20 550]]
              precision    recall  f1-score   support

           1       0.55      0.20      0.29       120
           2       0.85      0.96      0.90       570

    accuracy                           0.83       690
   macro avg       0.70      0.58      0.60       690
weighted avg       0.80      0.83      0.80       690

